In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


### <b>문제 2-1 : 콤마 구분 리스트 파서 활용</b>
<b>문제 설명</b><br>
: 사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.

<b>요구사항</b>
* CommaSeparatedListOutputParser 사용
* 사용자 입력을 받아 동적으로 처리
* 한국 관련 내용으로 추천



In [20]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=(
        "당신은 한국의 문화 전문가 입니다.\n"
        "사용자가 관심 있는 분야를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 추천해 주세요\n"
        "List five {category}.\n {format_instructions}"),
    input_variables=["category"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

PromptTemplate(input_variables=['category'], input_types={}, partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template='당신은 한국의 문화 전문가 입니다.\n사용자가 관심 있는 분야를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 추천해 주세요\nList five {category}.\n {format_instructions}')

In [21]:
# OpenAI 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | parser

# "사용자 입력"에 대한 체인 호출 실행
user_input = input("관심 있는 분야를 입력하세요 (예: 음식, 스포츠, 영화 등): ")
result = chain.invoke({"category": user_input})

# 쉼표로 구분된 리스트 출력
print(result)


['한국의 음식 분야를 입력하셨습니다. ', '다음은 한국의 유명한 음식 5가지입니다.', '비빔밥', '김치찌개', '불고기', '삼계탕', '잡채']


### <b>문제 2-2 : 영화 리뷰 감정 분석기</b>
<b>문제 설명</b><br>
: 영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

<b>요구사항</b>
* EnumOutputParser와 Enum 클래스 사용
* 여러 개의 테스트 리뷰로 검증
* 감정 분류 결과를 깔끔하게 출력

In [24]:
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 영화 리뷰를 감정으로 분류하는 감정 분석 전문가입니다.
다음 리뷰를 입력 받아 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

리뷰: "{review}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 리뷰
reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "시각효과는 좋았는데 스토리는 너무 엉성했어요.",
    "이 영화는 진짜 최고였어요. 또 보고 싶어요!"
]

print(f"테스트할 리뷰 {len(reviews)}개 준비 완료")

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 보통
모델 및 파서 설정 완료
테스트할 리뷰 5개 준비 완료


In [25]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(review, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"review": review})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(reviews)
    
    for i, review in enumerate(reviews, 1):
        print(f"\n{i}. 리뷰: {review}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(review, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(review, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

4. 리뷰: 시각효과는 좋았는데 스토리는 너무 엉성했어요.
   감정: 부정 

5. 리뷰: 이 영화는 진짜 최고였어요. 또 보고 싶어요!
   감정: 긍정 

=== 결과 요약 ===
성공: 5/5 (100.0%)
실패: 0/5


### <b>문제 2-3: 학생 정보 구조화 시스템</b>
<b>문제 설명</b><br>
: 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.

<b>요구사항</b>
* PydanticOutputParser와 BaseModel 사용
* 각 필드에 적절한 타입과 설명 추가
* 자유 형식의 텍스트에서 정보 추출

In [27]:
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

In [28]:
# 출력 구조를 정의하는 Pydantic 모델
class CoverLetter(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str]  = Field(description="취미")
    goal: str = Field(description="목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=CoverLetter)

# 프롬프트 템플릿 설정
template = """
다음 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "이름", "title": "Name", "type": "string"}, "age": {"description": "나이", "title": "Age", "type": "integer"}, "major": {"description": "전공", "title": "Major", "type": "string"}, "hobbies": {"description": "취미", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "목표", "title": "Goal", "type": "string"}}, "required": ["name", "age", "major", "hobbies", "goal"]}\n```'} messages=[Hu

In [30]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 체인 구성 및 실행
query = input("이름, 나이, 전공, 취미, 목표를 포함한 자기 소개를 입력해주세요.")
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")
    

이름: 정혜영
나이: 23
전공: 컴퓨터보안공학과
취미: 폰하기, 음악 듣기, 자기, 영화보기
목표: 훌륭한 개발자가 되고 싶어요


### <b>문제 2-4 : 여행 계획 분석기</b>
<b>문제 설명</b><br>
: 여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.
<b>요구사항</b>
* StructuredOutputParser와 ResponseSchema 사용
* 5개의 필드를 정의하여 정보 추출
* 자연어 텍스트에서 핵심 정보 파싱

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="기간"),
    ResponseSchema(name="budget", description="예산"),
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="activities", description="주요 활동 리스트")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행지
	"duration": string  // 기간
	"budget": string  // 예산
	"rating": string  // 5점 만점에서 예상 평점
	"activities": string  // 주요 활동 리스트
}
```


In [32]:
# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'activities': '해운대 바다 구경, 자갈치시장 회 먹기, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4'}
